In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

#####################################
# Libraries
#####################################
# Common libs
import pandas as pd
import numpy as np
import sys
import os
import random
from pathlib import Path

# Image processing
import imageio
import cv2
import skimage.transform
#from skimage.transform import rescale, resize, downscale_local_mean

# Charts
import matplotlib.pyplot as plt
import seaborn as sns

# ML, statistics
import scipy
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import confusion_matrix, roc_curve, auc, roc_auc_score

# Tensorflow
#from sklearn.preprocessing import OneHotEncoder
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Dense, Dropout, Flatten, Activation
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.optimizers import Adam, RMSprop

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

INPUT_DIR = "/kaggle/input/plant-pathology-2021-fgvc8/"
PRETRAINED_DIR = "/kaggle/input/pretrained-model/"

NUM_CLASSES = 6 #12
IMAGE_SIZE= (224,224)  #(256,256)

BATCH_SIZE = 32





In [2]:
class ModelTrainer:
    """
    Create and fit the model
    """
    
    def __init__(self):
        #(18632, 2) (17700, 2) (932, 2)
        #15930 , 1770, 932
        
        self.nb_train_samples = 15930 #4275 #32706
        self.nb_validation_samples = 1770 #475 #10902
        self.batch_size=512 #512 #32 #1024   #32
               
        
        self.img_width = IMAGE_SIZE[0]
        self.img_height = IMAGE_SIZE[1]
        print(self.img_width,self.img_height)
        
    
    def create_model(self, model_name):
        if model_name == "Xception" :
                model = self.create_model_1()
        elif model_name == "XceptionDenseNet121": 
                model = self.create_model_3()
        else:
                print("Model not found")
        self.model_name = model_name
        return model
    

    #Total params: 20,886,068
    def create_model_1(self):
        pretrained_model = tf.keras.applications.xception.Xception(include_top=False, weights='imagenet', input_shape=(self.img_width, self.img_height, 3))
        model = tf.keras.models.Sequential([
            pretrained_model,
            tf.keras.layers.GlobalAveragePooling2D(),
            tf.keras.layers.Dropout(0.3),
            tf.keras.layers.Dense(NUM_CLASSES,activation='softmax')
        ])
    
        model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
        #tf.keras.utils.plot_model(densenet_model, to_file='densenet_model.png')
        return model

    def create_model_3(self):
        path1 = PRETRAINED_DIR + 'xception_weights_tf_dim_ordering_tf_kernels_notop.h5'
        xception_model = tf.keras.models.Sequential([
           tf.keras.applications.xception.Xception(include_top=False, weights=path1, input_shape=(self.img_width, self.img_height, 3)),#(512, 512, 3)),
           tf.keras.layers.GlobalAveragePooling2D(),
           #tf.keras.layers.Dense(NUM_CLASSES,activation='softmax')
           tf.keras.layers.Dense(NUM_CLASSES,activation='sigmoid')
        ])
        # Freezing the weights
        #for layer in xception_model.layers[:-1]:
        #    layer.trainable=False
            
        #xception_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
        #xception_model.summary()
        path2 = PRETRAINED_DIR + 'densenet121_weights_tf_dim_ordering_tf_kernels_notop.h5'
        densenet_model = tf.keras.models.Sequential([
            tf.keras.applications.densenet.DenseNet121(include_top=False, weights=path2,input_shape=(self.img_width, self.img_height, 3)),#(512, 512, 3)),
            tf.keras.layers.GlobalAveragePooling2D(),
            #tf.keras.layers.Dense(NUM_CLASSES,activation='softmax')
            tf.keras.layers.Dense(NUM_CLASSES,activation='sigmoid')
        ])
        #for layer in densenet_model.layers[:-1]:
        #    layer.trainable=False
        #densenet_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
        #densenet_model.summary()
        
        inputs = tf.keras.Input(shape=(self.img_width, self.img_height, 3)) #(512, 512, 3))

        xception_output = xception_model(inputs)
        densenet_output = densenet_model(inputs)

        outputs = tf.keras.layers.average([densenet_output, xception_output])


        model = tf.keras.Model(inputs=inputs, outputs=outputs)
        return model

In [3]:
trainer = ModelTrainer()
model = trainer.create_model("XceptionDenseNet121") # 0.9062 Total params: 27,935,872
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.load_weights('/kaggle/input/pretrained-model/best_XceptionDenseNet121_v2.h5')


224 224


In [4]:
from keras.preprocessing.image import ImageDataGenerator



test_dir = '/kaggle/input/plant-pathology-2021-fgvc8/test_images/'
test_df = pd.DataFrame()
test_df['image'] = os.listdir(test_dir)

IMSIZES = (224, 240, 260, 300, 380, 456, 528, 600)
im_size = IMSIZES[7]

testgen = ImageDataGenerator(
            #preprocessing_function = tf.keras.applications.xception.preprocess_input,
            horizontal_flip=True,
            vertical_flip=True,
            brightness_range=(0.8, 1.2),
            rescale=1/255.0)



In [5]:
#classes = ['complex', 'frog_eye_leaf_spot', 'frog_eye_leaf_spot complex', 'healthy', 'powdery_mildew', 'powdery_mildew complex', 'rust', 'rust complex', 'rust frog_eye_leaf_spot', 'scab', 'scab frog_eye_leaf_spot', 'scab frog_eye_leaf_spot complex']
classes = ['complex', 'frog_eye_leaf_spot', 'healthy', 'powdery_mildew', 'rust', 'scab'] #,'frog_eye_leaf_spot complex','powdery_mildew complex', 'rust complex', 'rust frog_eye_leaf_spot', 'scab frog_eye_leaf_spot', 'scab frog_eye_leaf_spot complex']

class Submitter:
    """
    Predict and submit
    """
    def __init__(self, model, img_size):
        self.model = model
        batch_size=BATCH_SIZE
        print("Initializing submitter")
        #Submission generator
        # flow_from_directory for input/test didn't work for me, so quick fix is to use flow_from_dataframe with list of files
        # Load list of files from test folder into dataframe
        self.test_files_df=pd.DataFrame()
        TEST_DIR = INPUT_DIR + 'test_images/'
        self.test_files_df['image']=os.listdir(TEST_DIR)
        print("Loaded test files list")
        
        
        # Create generator in it
        #self.generator=ImageDataGenerator(rescale=1./255.).flow_from_dataframe(
        _test_datagen=ImageDataGenerator(rescale=1./255.)
        self.generator = _test_datagen.flow_from_dataframe(
                    dataframe=self.test_files_df,
                    directory=TEST_DIR,
                    x_col="image",
                    y_col=None,
                    #has_ext=True,
                    class_mode=None,
                    batch_size=batch_size,
                    seed=42,
                    shuffle=False,
                    target_size=img_size)    
        
        print('Submission generator created')    


    def predict_for_submit(self):
        """
        Predict submission test data and form dataframe to submit
        """
        print("Forming submission dataframe...")
        # Predict
        y_pred = self.model.predict(self.generator)
        y_pred = y_pred.tolist()
        print(y_pred)
        #y_pred = np.argmax(y_pred, axis=1)
        #print(y_pred)
        
        indices = []
        for pred in y_pred:
            temp = []
            for category in pred:
                #print(category)
                if category>=0.3:
                    temp.append(pred.index(category))
            #print(temp)
            if temp!=[]:
                indices.append(temp)
            else:
                temp.append(np.argmax(pred))
                indices.append(temp)
        
        print(indices)
        
        y_pred_str = []        
        for image in indices:
            temp = []
            for i in image:
                #if i > 5:
                #    temp.append(classes[0])
                #else:
                temp.append(classes[i])
                    
            y_pred_str.append(' '.join(temp))

        print(y_pred_str)
        
        self.test_files_df['labels'] = y_pred_str
        # Write to csv
        self.test_files_df.to_csv('./submission.csv', index=False)
        print("Submission completed: written submission.csv")
        return self.test_files_df

In [6]:
# Get dataframe for submission
submitter = Submitter(model, IMAGE_SIZE)
submission_df = submitter.predict_for_submit()     
submission_df.head()

Initializing submitter
Loaded test files list
Found 3 validated image filenames.
Submission generator created
Forming submission dataframe...
[[0.6049584150314331, 0.5035156011581421, 8.621523193141911e-06, 0.0006842989241704345, 0.0008050501346588135, 0.47748398780822754], [0.07540777325630188, 0.9548231363296509, 6.363177089951932e-05, 4.355760393082164e-05, 0.006044939160346985, 0.01754032075405121], [0.008406117558479309, 0.0027231425046920776, 0.0016592990141361952, 0.00042550344369374216, 0.001295715570449829, 0.9818695783615112]]
[[0, 1, 5], [1], [5]]
['complex frog_eye_leaf_spot scab', 'frog_eye_leaf_spot', 'scab']
Submission completed: written submission.csv


,image,labels
0,ad8770db05586b59.jpg,complex frog_eye_leaf_spot scab
1,c7b03e718489f3ca.jpg,frog_eye_leaf_spot
2,85f8cb619c66b863.jpg,scab


In [7]:
#!cp /kaggle/input/notebook-sam/best_XceptionDenseNet121.h5 .

#import os
#print(os.listdir("/kaggle/input/"))
#os.chdir(r'/kaggle/working')
#from IPython.display import FileLink
#FileLink(r'./best_XceptionDenseNet121.h5')